In [1]:
import numpy as np
import scipy.sparse as sp
import graphgallery
from graphgallery import functional as gf 
graphgallery.__version__

'1.1.0'

# Load datasets
+ cora
+ citeseer
+ cora_ml
+ pubmed

In [2]:
dataset = 'cora'
data = graphgallery.datasets.NPZDataset(dataset, root='~/GraphData/datasets/', transform="standardize",  verbose=True)
splits = data.split_nodes(random_state=15)
graph = data.graph

Dataset 'cora' has already existed, loading it...
Processing dataset 'cora'...
Processing completed.


# MedianGCN and TrimmedGCN

In [3]:
graphgallery.set_backend('torch')

PyTorch 1.9.0+cu102 Backend

In [4]:
from graphgallery.gallery.nodeclas import MedianGCN
from graphgallery.gallery.nodeclas import TrimmedGCN
from graphgallery.gallery import callbacks

In [5]:
median = MedianGCN(device="cuda", seed=10).setup_graph(graph).build()
# median = TrimmedGCN(device="cuda", seed=10).setup_graph(graph).build()
cb = callbacks.ModelCheckpoint('model.pth', monitor='val_accuracy')
median.fit(splits.train_nodes, splits.val_nodes, verbose=1, callbacks=[cb])

# performance on test set
median.evaluate(splits.test_nodes)

Training...
100/100 [==============================] - Total: 91.06s - 910ms/step- loss: 0.194 - accuracy: 0.931 - val_loss: 0.633 - val_accuracy: 0.823
Testing...
1/1 [==============================] - Total: 239.38ms - 239ms/step- loss: 0.548 - accuracy: 0.843


Objects in BunchDict:
╒══════════╤═══════════╕
│ Names    │   Objects │
╞══════════╪═══════════╡
│ loss     │  0.548302 │
├──────────┼───────────┤
│ accuracy │  0.842555 │
╘══════════╧═══════════╛

In [6]:
median.model

MedianGCN(
  (conv): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): MedianConv(1433, 16)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): MedianConv(16, 7)
  )
)

# Baseline: GCN

In [7]:
gcn = graphgallery.gallery.nodeclas.GCN(device="cuda", seed=10).setup_graph(graph).build()
cb = callbacks.ModelCheckpoint('model.pth', monitor='val_accuracy')
gcn.fit(splits.train_nodes, splits.val_nodes, verbose=1, callbacks=[cb])

# performance on test set
gcn.evaluate(splits.test_nodes)

Training...
100/100 [==============================] - Total: 428.46ms - 4ms/step- loss: 0.101 - accuracy: 0.98 - val_loss: 0.6 - val_accuracy: 0.807
Testing...
1/1 [==============================] - Total: 1.84ms - 1ms/step- loss: 0.557 - accuracy: 0.842


Objects in BunchDict:
╒══════════╤═══════════╕
│ Names    │   Objects │
╞══════════╪═══════════╡
│ loss     │  0.557011 │
├──────────┼───────────┤
│ accuracy │  0.841549 │
╘══════════╧═══════════╛

In [8]:
gcn.model

GCN(
  (conv): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): GCNConv(1433, 16)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): GCNConv(16, 7)
  )
)